In [2]:
import pandas as pd
import os
import os.path as osp
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import collections
import random
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# --- load relevant entities from the LP dataset ---
# ilpc small
lp_path = '../data/ilpc/raw/small'
# ilpc large
#lp_path = '../data/ilpc/raw/large'
# wikidata5m_inductive
#lp_path = '../data/wikidata5m_inductive'
# FB15k-237
#lp_path = '../data/fb15k-237'


lp_files = [x for x in os.listdir(lp_path) if x.endswith('.txt')]
print('dataset files:', lp_files)
lp_entities = []
lp_relations = []
for file in lp_files:
    triple_table = pd.read_csv(osp.join(lp_path, file), delimiter='\t', header=None)
    lp_entities.extend(triple_table[0].tolist() + triple_table[2].tolist())
    lp_relations.extend(triple_table[1].tolist())

entity_mentions = collections.Counter(lp_entities)
entity_mention_counts = sorted(entity_mentions.values(), reverse=True)

lp_entities = set(lp_entities)
lp_relations = set(lp_relations)
print(f'num entities of {lp_path}:', len(lp_entities))
print(f'num relations of {lp_path}:', len(lp_relations))

# lp_entities, lp_relations

dataset files: ['inference_validation.txt', 'train.txt', 'inference.txt', 'inference_test.txt']
num entities of ../data/ilpc/raw/small: 16883
num relations of ../data/ilpc/raw/small: 48


In [4]:
graph_type = 'triples'  # triples or page_links or all

trex_triple_table = pd.read_csv(f'../data/corpus_graphs/trex_{graph_type}.txt', delimiter='\t', header=None)
trex_entities = trex_triple_table[0].tolist() + trex_triple_table[2].tolist()
trex_relations = set(trex_triple_table[1].tolist())
k = 1  # k-hop
degree_filter = 0
ratio_to_maintain = 0.5

In [5]:
# todo adjust node degree to the relevant relations
# compute degree for each entity in our trex graph
relevant_relations = set(list(lp_relations)) #  + ['PageLink']
print(relevant_relations)

trex_triples = list(trex_triple_table.itertuples(index=False, name=None))
print(trex_triples)

entity_mentions = {e: 0 for e in set(trex_entities)}
for triple in tqdm(trex_triples):
    if triple[1] in relevant_relations or all_relations:
        entity_mentions[triple[0]] += 1
        entity_mentions[triple[2]] += 1

print(np.array(list(entity_mentions.values())).mean())


covered_entities = lp_entities.copy()
print(len(covered_entities))

k_counter = k
while k_counter:
    covered_entities_new = []
    for triple in tqdm(trex_triples):
        if triple[0] in covered_entities:
            if entity_mentions[triple[0]] < degree_filter:
                if triple[1] in relevant_relations:
                    if random.uniform(0, 1.0) > ratio_to_maintain:
                        covered_entities_new.extend([str(triple[0]), str(triple[2])])
        if triple[2] in covered_entities:
            if entity_mentions[triple[2]] < degree_filter:
                if triple[1] in relevant_relations:
                    if random.uniform(0, 1.0) > ratio_to_maintain:
                        covered_entities_new.extend([str(triple[0]), str(triple[2])])

    covered_entities = set(covered_entities).union(covered_entities_new)
    print(len(covered_entities))
    k_counter -= 1

print('Entities maintained:', len(covered_entities)/len(set(trex_entities)))
print(len(set(covered_entities).intersection(set(trex_entities))))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 5410928/5410928 [00:04<00:00, 1190746.62it/s]


2.5703380820903514
16883


100%|██████████| 5410928/5410928 [00:02<00:00, 2576187.05it/s]


54844
Entities maintained: 0.019448461435350638
53326


In [ ]:
triples_filtered = []
test_graph = '../data/ilpc/raw/small/inference_test.txt'
test_triple_df = pd.read_csv(test_graph, delimiter='\t', header=None)
test_triples = list(test_triple_df.itertuples(index=False, name=None))

print(len(test_triples))

for triple in tqdm(trex_triples):
        if triple[0] in covered_entities and triple[2] in covered_entities and (all_relations or triple[1] in relevant_relations):
            if triple not in test_triples:
                triples_filtered.append(triple)

print('number of triples:', len(triples_filtered))
print('Triples maintained:', len(triples_filtered)/len(trex_triples))